第六章代码总结：

本章通过选取阈值点，对图像主体与背景进行分割。分为手动与自动。

自动：直方图技术法，熵算法（太麻烦了），Otsu（全局法中最棒），自适应阈值法（无敌）

In [ ]:
#手动阈值
thresh = 150#阈值
maxVal = 255#输出主体的像素值
type = cv2.THRESH_BINARY#用0替代，大于阈值输出maxVal，小于等于输出0；THRESH_BINARY_INV=1相反
dst = cv2.threshold(src,thresh,maxVal,type)

# 自动阈值 效果Otsu > Triangle > histogram = 熵 
#Otsu(只支持8位图)
otsu_thresh = 0
otsu_thresh,dst_otsu = cv2.threshold(src,otsu_thresh,255,8)#最后的8,代表type为cv2.THRESH_OTSU

#Triangle三角形法（直方图法原理同三角形法）
triangle_thresh = 0
triangle_thresh,dst_tri = cv2.threshold(src,triangle_thresh,255,16)

#直方图histogram与熵算法效果几乎相同，过程太复杂，想看原理的看6.2，6.3

自适应阈值法

全局法对光照不均的图像效果不好，根据每个像素点位置及其领域来设置每一像素点的阈值就是自适应阈值法。

先需对图像进行平滑处理：均值（mean），高斯（gaussian），中值（median）

计算像素领域灰度均值乘以比例系数（1-ratio），ratio一般指定0.15。

结果作为阈值参考值。平滑算子高宽为基数，须大于被识别主体宽度保证效果。

In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
#平滑处理采用均值平滑(cv2.blur或者cv2.boxFilter)
def adaptiveThresh(I,winSize,ratio=0.15):#比例系数ratio一般0.15
    I_mean = cv2.blur(I,winSize)
    #I_mean = cv2.boxFilter(I,cv2.CV_32FC1,winSize)#结果相同,必须指定32位
    out = I - (1.0-ratio)*I_mean
    out[out>=0] = 255
    out[out<0] = 0
    out = out.astype(np.uint8)#图像格式转换为255，
    return out

src = cv2.imread("image3.png",0)
dst = adaptiveThresh(src,(7,7),0.15)

#PythonAPI，效果输出没上面编写的好，奇怪，可能是我src没预先平滑好？评论大佬解答下
#7代表平滑算子（7，7），0.15为ratio(不懂的，就在上段啊喂，别跳)，中值平滑adaptive.Threshold没给预设
dst_adaptive_mean = cv2.adaptiveThreshold(src,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,7,0.15)
dst_adaptive_gaussian = cv2.adaptiveThreshold(src,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,7,0.15)


最后6.6介绍二值图的逻辑运算，要求图像格式相同

dst_and = cv2.bitwise_and(src1,src2)#与运算

dst_or = cv2.bitwise_or(src1,src2)#或运算